[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/structural-break/quickstarters/baseline/baseline.ipynb)

![Banner](https://raw.githubusercontent.com/crunchdao/quickstarters/refs/heads/master/competitions/structural-break/assets/banner.webp)

# ADIA Lab Structural Break Challenge

## Challenge Overview

Welcome to the ADIA Lab Structural Break Challenge! In this challenge, you will analyze univariate time series data to determine whether a structural break has occurred at a specified boundary point.

### What is a Structural Break?

A structural break occurs when the process governing the data generation changes at a certain point in time. These changes can be subtle or dramatic, and detecting them accurately is crucial across various domains such as climatology, industrial monitoring, finance, and healthcare.

![Structural Break Example](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/competitions/structural-break/quickstarters/baseline/images/example.png)

### Your Task

For each time series in the test set, you need to predict a score between `0` and `1`:
- Values closer to `0` indicate no structural break at the specified boundary point;
- Values closer to `1` indicate a structural break did occur.

### Evaluation Metric

The evaluation metric is [ROC AUC (Area Under the Receiver Operating Characteristic Curve)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html), which measures the performance of detection algorithms regardless of their specific calibration.

- ROC AUC around `0.5`: No better than random chance;
- ROC AUC approaching `1.0`: Perfect detection.

# Setup

The first steps to get started are:
1. Get the setup command
2. Execute it in the cell below

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Reveal token](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/reveal-token.gif)

In [1]:
%pip install crunch-cli --upgrade --quiet --progress-bar off
!crunch setup-notebook structural-break 9KnBOg5xU4qvMxIOrbROU6Ou

crunch-cli, version 6.5.0
you appear to have never submitted code before
data/X_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/y_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
                                
---
Success! Your environment has been correctly setup.
Next recommended actions:
1. Load the Crunch Toolings: `crunch = crunch.load_notebook()`
2. Execute the cells with your code
3. Run a test: `crunch.test()`
4. Download and submit your code to the p

In [14]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import t
import time
from multiprocessing import Pool, cpu_count
from scipy.stats import shapiro
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import skew, kurtosis, iqr, ttest_ind, ks_2samp
# from pykalman import KalmanFilter
import scipy

import joblib
import typing

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin


# ---------------- Custom Transformer ----------------
from multiprocessing import Pool, cpu_count

# Your model

## Setup

In [3]:
import os
import typing

# Import your dependencies
import joblib
import pandas as pd
import scipy
import sklearn.metrics

In [4]:
import crunch

# Load the Crunch Toolings
crunch = crunch.load_notebook()

loaded inline runner with module: <module '__main__'>

cli version: 6.5.0
available ram: 12.67 gb
available cpu: 2 core
----


## Understanding the Data

The dataset consists of univariate time series, each containing ~2,000-5,000 values with a designated boundary point. For each time series, you need to determine whether a structural break occurred at this boundary point.

The data was downloaded when you setup your local environment and is now available in the `data/` directory.

In [5]:
# Load the data simply
X_train, y_train, X_test = crunch.load_data()

data/X_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/X_test.reduced.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
data/y_test.reduced.parquet: already exists, file length match


### Understanding `X_train`

The training data is structured as a pandas DataFrame with a MultiIndex:

**Index Levels:**
- `id`: Identifies the unique time series
- `time`: The timestep within each time series

**Columns:**
- `value`: The actual time series value at each timestep
- `period`: A binary indicator where `0` represents the **period before** the boundary point, and `1` represents the **period after** the boundary point

In [ ]:
X_train

value  period
id    time                  
0     0     0.001858       0
      1    -0.001664       0
      2    -0.004386       0
      3     0.000699       0
      4    -0.002433       0
...              ...     ...
10000 1890 -0.005903       1
      1891  0.007295       1
      1892  0.003527       1
      1893  0.007218       1
      1894  0.000034       1

[23802099 rows x 2 columns]

### Understanding `y_train`

This is a simple `pandas.Series` that tells if a dataset id has a structural breakpoint or not.

**Index:**
- `id`: the ID of the dataset

**Value:**
- `structural_breakpoint`: Boolean indicating whether a structural break occurred (`True`) or not (`False`)

### Understanding `X_test`

The test data is provided as a **`list` of `pandas.DataFrame`s** with the same format as [`X_train`](#understanding-X_test).

It is structured as a list to encourage processing records one by one, which will be mandatory in the `infer()` function.

## Strategy Implementation

There are multiple approaches you can take to detect structural breaks:

1. **Statistical Tests**: Compare distributions before and after the boundary point;
2. **Feature Engineering**: Extract features from both segments for comparison;
3. **Time Series Modeling**: Detect deviations from expected patterns;
4. **Machine Learning**: Train models to recognize break patterns from labeled examples.

The baseline implementation below uses a simple statistical approach: a t-test to compare the distributions before and after the boundary point.

### The `train()` Function

In this function, you build and train your model for making inferences on the test data. Your model must be stored in the `model_directory_path`.

The baseline implementation below doesn't require a pre-trained model, as it uses a statistical test that will be computed at inference time.

In [12]:
indices_x = X_train.index.get_level_values(0).unique().tolist()
indices_x.sort()

In [77]:


def kalman_stats(series):
    kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
    # fewer EM iterations
    kf = kf.em(series, n_iter=1)
    state_means, _ = kf.smooth(series)
    state_means = state_means.flatten()

    smoothed_mean = np.mean(state_means)
    smoothed_std = np.std(state_means)
    residual = series - state_means
    residual_std = np.std(residual)
    trend_strength = smoothed_std / (np.std(series) + 1e-6)

    return {
        'kf_mean': smoothed_mean,
        'kf_std': smoothed_std,
        'kf_residual_std': residual_std,
        'kf_trend_strength': trend_strength
    }

def fast_ema(x, span):
    alpha = 2/(span+1)
    ema = np.empty_like(x)
    ema[0] = x[0]
    for i in range(1, len(x)):
        ema[i] = alpha * x[i] + (1-alpha) * ema[i-1]
    return ema[-1]

def extract_features(a, b):
    features = {}

    # Basic statistics
    features['mean_a'] = np.mean(a)
    features['mean_b'] = np.mean(b)
    features['std_a'] = np.std(a)
    features['std_b'] = np.std(b)
    features['skew_a'] = skew(a)
    features['skew_b'] = skew(b)
    features['kurt_a'] = kurtosis(a)
    features['kurt_b'] = kurtosis(b)
    features['iqr_a'] = iqr(a)
    features['iqr_b'] = iqr(b)
    features['std_ratio'] = features['std_b'] / (features['std_a'] + 1e-10)  # avoid div0

    # Fit t-distribution (consider sampling if large arrays)
    params_a = scipy.stats.t.fit(a)
    params_b = scipy.stats.t.fit(b)
    features['t_val_a'] = params_a[0]
    features['t_mu_a'] = params_a[1]
    features['t_sigma_a'] = params_a[2]
    features['t_val_b'] = params_b[0]
    features['t_mu_b'] = params_b[1]
    features['t_sigma_b'] = params_b[2]

    # Differences
    features['mean_diff'] = features['mean_b'] - features['mean_a']
    features['std_diff'] = features['std_b'] - features['std_a']

    # Statistical tests
    features['ttest_p'] = ttest_ind(a, b, equal_var=False).pvalue
    features['ks_p'] = ks_2samp(a, b).pvalue

    # Kalman features
    # try:

    #   kf_a = kalman_stats(np.array(a))
    #   kf_b = kalman_stats(np.array(b))
    #   for k, v in kf_a.items():
    #       features[f'{k}_a'] = v
    #   for k, v in kf_b.items():
    #       features[f'{k}_b'] = v
    #   features['kf_trend_strength_diff'] = features['kf_trend_strength_b'] - features['kf_trend_strength_a']

    # except:
    #   pass

    # EMA features (use fast_ema)
    ema_windows = [5, 10, 15, 20, 25]
    for w in ema_windows:
        ema_a = fast_ema(a, w)
        ema_b = fast_ema(b, w)
        features[f'ema{w}_a'] = ema_a
        features[f'ema{w}_b'] = ema_b
        features[f'ema{w}_diff'] = ema_b - ema_a

    # Cross-correlation
    min_len = min(len(a), len(b))
    a = a[:min_len]
    b = b[:min_len]
    max_lag = min(10, min_len - 1)
    xcorr = []
    for lag in range(1, max_lag + 1):
        corr = np.corrcoef(a[:-lag], b[lag:])[0, 1]
        xcorr.append(corr)

    features['max_xcorr'] = np.nanmax(xcorr)
    features['mean_xcorr'] = np.nanmean(xcorr)

    return features


In [78]:
def process_chunk(temp):
  t0_series = temp[temp.period == 0].value.values
  t1_series = temp[temp.period == 1].value.values
  features_v = extract_features(t0_series, t1_series)
  return features_v



def train(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    model_directory_path: str,
):
    os.makedirs(model_directory_path, exist_ok=True)

    # try:
    #   !pip install pykalman
    # except:
    #   pass



    # For our baseline t-test approach, we don't need to train a model
    # This is essentially an unsupervised approach calculated at inference time
    data_file = os.path.join(model_directory_path,"data.csv")

      index_x = X_train.index.get_level_values(0).unique().tolist()
      features_list = []
      for i in index_x:
        if i%100 == 0:
          print(f"i am running {i} for feature extraction")
          features_list.append(process_chunk(X_train.loc[i]))

      data = pd.DataFrame.from_records(features_list,index=index_x)

      data.index.name = "time"
      # data.to_csv(data_file)

    base_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='auc',
    random_state=42,

)

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', base_model)
    ])

    param_grid = {
        'clf__n_estimators': [100, 200],
        'clf__max_depth': [3, 5, 10],
        'clf__learning_rate': [0.01, 0.1,0.2],
        'clf__subsample': [0.8, 1.0],
        'clf__colsample_bytree': [0.8, 1.0]
    }

    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring='roc_auc',
        cv=3,
        n_jobs=-1,
        verbose=1
    )

    grid.fit(data, y_train)

    best_model = grid.best_estimator_
    os.makedirs(model_directory_path, exist_ok=True)
    #best_model.save_model('model.json')
    joblib.dump(best_model, os.path.join(model_directory_path, 'model.joblib'))

### The `infer()` Function

In the inference function, your trained model (if any) is loaded and used to make predictions on test data.

**Important workflow:**
1. Load your model;
2. Use the `yield` statement to signal readiness to the runner;
3. Process each dataset one by one within the for loop;
4. For each dataset, use `yield prediction` to return your prediction.

**Note:** The datasets can only be iterated once!

In [80]:
# def infer(
#     X_test: typing.Iterable[pd.DataFrame],
#     model_directory_path: str,
# ):
#     model = joblib.load(os.path.join(model_directory_path, 'model.joblib'))

#     yield  # Mark as ready

#     # X_test can only be iterated once.
#     # Before getting the next dataset, you must predict the current one.
#     for dataset in X_test:
#         # Baseline approach: Compute t-test between values before and after boundary point
#         # The negative p-value is used as our score - smaller p-values (larger negative numbers)
#         # indicate more evidence against the null hypothesis that distributions are the same,
#         # suggesting a structural break
#         def t_test(u: pd.DataFrame):
#             return -scipy.stats.ttest_ind(
#                 u["value"][u["period"] == 0],  # Values before boundary point
#                 u["value"][u["period"] == 1],  # Values after boundary point
#             ).pvalue

#         prediction = t_test(dataset)
#         yield prediction  # Send the prediction for the current dataset

#         # Note: This baseline approach uses a t-test to compare the distributions
#         # before and after the boundary point. A smaller p-value (larger negative number)
#         # suggests stronger evidence that the distributions are different,
#         # indicating a potential structural break.


def infer(
    X_test: typing.Iterable[pd.DataFrame],
    model_directory_path: str,
):
    # try:
    #   !pip install pykalman
    # except:
    #   pass
    # Load the trained model
    model = joblib.load(os.path.join(model_directory_path, 'model.joblib'))

    yield  # Mark as ready for inference

    for dataset in X_test:
        # Feature extraction using the same logic as in training
        features = process_chunk(dataset)
        features_df = pd.DataFrame([features])

        # Predict probability of the positive class (1)
        prediction = model.predict_proba(features_df)[0, 1]

        yield prediction


## Local testing

To make sure your `train()` and `infer()` function are working properly, you can call the `crunch.test()` function that will reproduce the cloud environment locally. <br />
Even if it is not perfect, it should give you a quick idea if your model is working properly.

In [81]:
crunch.test(
)

ignoring cell #19: invalid syntax (<unknown>, line 2)


23:55:30 forbidden library: pykalman
23:55:30 forbidden library: google
23:55:30 
23:55:30 started
23:55:30 running local test
23:55:30 internet access isn't restricted, no check will be done
23:55:30 
23:55:31 starting unstructured loop...
23:55:31 executing - command=infer


data/X_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/X_test.reduced.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
data/y_test.reduced.parquet: already exists, file length match


23:55:58 checking determinism by executing the inference again with 30% of the data (tolerance: 1e-08)
23:55:58 executing - command=infer
23:56:09 determinism check: passed
23:56:09 save prediction - path=data/prediction.parquet
23:56:09 ended
23:56:09 duration - time=00:00:38
23:56:09 memory - before="1.41 GB" after="1.41 GB" consumed="4.6 MB"


## Results

Once the local tester is done, you can preview the result stored in `data/prediction.parquet`.

In [ ]:
prediction = pd.read_parquet("data/prediction.parquet")
prediction

### Local scoring

You can call the function that the system uses to estimate your score locally.

In [ ]:
# Load the targets
target = pd.read_parquet("data/y_test.reduced.parquet")["structural_breakpoint"]

# Call the scoring function
sklearn.metrics.roc_auc_score(
    target,
    prediction,
)

# Submit your Notebook

To submit your work, you must:
1. Download your Notebook from Colab
2. Upload it to the platform
3. Create a run to validate it

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Download and Submit Notebook](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/download-and-submit-notebook.gif)